In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras import mixed_precision


In [4]:
emotions = ["aloevera", "banana", "bilimbi", "cantaloupe", "cassava", "coconut", "corn", "cucumber",
            "curcuma", "eggplant", "galangal", "ginger", "guava", "kale", "longbeans", "mango", "melon",
            "orange", "paddy", "papaya", "peper chili", "pineapple", "pomelo", "shallot", "soybeans",
            "spinach", "sweet potatoes", "tobacco", "waterapple", "watermelon"]
width = 128
height = 128
path_train_folder = r'data\split_ttv_dataset_type_of_plants\Train_Set_Folder'
path_validation_folder = r'data\split_ttv_dataset_type_of_plants\Validation_Set_Folder' 
path_test_folder = r'data\split_ttv_dataset_type_of_plants\Test_Set_Folder'
num_labels = len(emotions)

# Enable mixed precision
mixed_precision.set_global_policy('mixed_float16')

# Define paths
train_dir = r'data\split_ttv_dataset_type_of_plants\Train_Set_Folder'
validation_dir = r'data\split_ttv_dataset_type_of_plants\Validation_Set_Folder' 
test_dir = r'data\split_ttv_dataset_type_of_plants\Test_Set_Folder'

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 30 # 1081


In [3]:
# Create datasets
train_dataset = image_dataset_from_directory(
	train_dir,
	labels='inferred',
	label_mode='categorical',
	batch_size=BATCH_SIZE,
	image_size=IMG_SIZE,
	shuffle=True,
	seed=123
)

validation_dataset = image_dataset_from_directory(
	validation_dir,
	labels='inferred',
	label_mode='categorical',
	batch_size=BATCH_SIZE,
	image_size=IMG_SIZE,
	shuffle=True,
	seed=123
)

test_dataset = image_dataset_from_directory(
	test_dir,
	labels='inferred',
	label_mode='categorical',
	batch_size=BATCH_SIZE,
	image_size=IMG_SIZE,
	shuffle=False
)

# Prefetch
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Compute class weights
train_labels = []
for _, labels in train_dataset.unbatch():
	train_labels.append(np.argmax(labels.numpy()))

class_weights_values = class_weight.compute_class_weight(
	class_weight='balanced',
	classes=np.unique(train_labels),
	y=train_labels
)

class_weights = {i: weight for i, weight in enumerate(class_weights_values)}


Found 23972 files belonging to 30 classes.
Found 3030 files belonging to 30 classes.
Found 2998 files belonging to 30 classes.


In [5]:
# Data Augmentation
data_augmentation = Sequential([
	RandomFlip("horizontal_and_vertical"),
	RandomRotation(0.2),
	RandomZoom(0.2),
])

# Build the model
base_model = EfficientNetB0(
	weights='imagenet',
	include_top=False,
	input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [6]:
# Create model with data augmentation
input_layer = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = data_augmentation(input_layer)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
output_layer = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = models.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(
	optimizer=optimizers.Adam(learning_rate=1e-4),
	loss='categorical_crossentropy',
	metrics=['accuracy']
)

In [9]:
# Callbacks
early_stopping = EarlyStopping(
	monitor='val_accuracy',
	patience=10,
	restore_best_weights=True
)

checkpoint = ModelCheckpoint(
	'best_model.keras',
	monitor='val_accuracy',
	save_best_only=True,
	verbose=1
)

reduce_lr = ReduceLROnPlateau(
	monitor='val_accuracy',
	factor=0.2,
	patience=5,
	verbose=1,
	min_lr=1e-7
)

callbacks = [early_stopping, checkpoint, reduce_lr]


In [10]:
# Train the model
EPOCHS = 30
history = model.fit(
	train_dataset,
	epochs=EPOCHS,
	validation_data=validation_dataset,
	callbacks=callbacks,
	class_weight=class_weights
)


Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1477 - loss: 3.7786
Epoch 1: val_accuracy improved from -inf to 0.67987, saving model to best_model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 1352s 2s/step - accuracy: 0.1479 - loss: 3.7775 - val_accuracy: 0.6799 - val_loss: 1.1112 - learning_rate: 1.0000e-04
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5151 - loss: 1.6657
Epoch 2: val_accuracy improved from 0.67987 to 0.79241, saving model to best_model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 1710s 2s/step - accuracy: 0.5152 - loss: 1.6656 - val_accuracy: 0.7924 - val_loss: 0.7064 - learning_rate: 1.0000e-04
Epoch 3/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6416 - loss: 1.2085
Epoch 3: val_accuracy improved from 0.79241 to 0.82805, saving model to best_model.keras
750/750 ━━━━━━━━━━━━━━━━━━━━ 1745s 2s/step - accuracy: 0.6416 - loss: 1.2084 - val_accuracy: 0.8281 - val_loss: 0.5560 - learning_rate: 1.0000e-04
Epoch 4/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Fine-tuning
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20

for layer in base_model.layers[:fine_tune_at]:
	layer.trainable = False

model.compile(
	optimizer=optimizers.Adam(learning_rate=1e-5),
	loss='categorical_crossentropy',
	metrics=['accuracy']
)

fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
	train_dataset,
	epochs=total_epochs,
	initial_epoch=history.epoch[-1],
	validation_data=validation_dataset,
	callbacks=callbacks,
	class_weight=class_weights
)


In [12]:
# Load the best model
#model.load_weights('best_model.keras')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


94/94 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9411 - loss: 0.1672
Test Accuracy: 94.40%


In [ ]:
# Classification report
y_pred = []
y_true = []

for images, labels in test_dataset:
	preds = model.predict(images)
	y_pred.extend(np.argmax(preds, axis=1))
	y_true.extend(np.argmax(labels.numpy(), axis=1))

report = classification_report(y_true, y_pred, target_names=train_dataset.class_names)
print(report)


In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)


In [ ]:
# Function to plot misclassified images
def plot_misclassified(images, true_labels, pred_labels, class_names, num=5):
	plt.figure(figsize=(15, 15))
	for i in range(num):
		plt.subplot(1, num, i+1)
		plt.imshow(images[i].astype("uint8"))
		plt.title(f"True: {class_names[true_labels[i]]}\nPred: {class_names[pred_labels[i]]}")
		plt.axis('off')
	plt.show()


In [ ]:
# Collect misclassified examples
misclassified_images = []
misclassified_true = []
misclassified_pred = []

for images, labels in test_dataset:
	preds = model.predict(images)
	preds = np.argmax(preds, axis=1)
	true = np.argmax(labels.numpy(), axis=1)
	for img, t, p in zip(images, true, preds):
		if t != p:
			misclassified_images.append(img.numpy())
			misclassified_true.append(t)
			misclassified_pred.append(p)
		if len(misclassified_images) >= 5:
			break
		if len(misclassified_images) >= 5:
			break

# Plot misclassified images
plot_misclassified(
	misclassified_images,
	misclassified_true,
	misclassified_pred,
	train_dataset.class_names,
	num=5
)


In [ ]:
# Save the model
model.save('efficientnet_plant_classifier.h5')